In [17]:
import urllib
import json
import pandas as pd

In [18]:
# Secret token
token = open("ACCESS_TOKEN.txt", "r").read()

In [91]:
# Format a request URI for the Genius API
search_term = 'Ayla Celik'
_URL_API = "https://api.genius.com/"
_URL_SEARCH = "search?q="
querystring = _URL_API + _URL_SEARCH + urllib.parse.quote(search_term)
request = urllib.request.Request(querystring)
request.add_header("Authorization", "Bearer " + token)
request.add_header("User-Agent", "")

In [92]:
df = pd.read_csv('song_ids.txt', sep=',', names=['artist_id', 'song_id'])
df.head()

,artist_id,song_id
0,1421,3477888
1,1421,330374
2,1421,196277
3,1421,54903
4,1421,1933500


In [310]:
# Format the search query using the 'referents' api
# Given a song id, returns all of its referents (lines that have been annotated)
# and the corresponding annotations.
song_id = int(df.sample().song_id.values[0])
print(song_id)
querystring = "https://api.genius.com/referents?song_id=" + str(song_id) + "&per_page=50"
request = urllib.request.Request(querystring)
request.add_header("Authorization", "Bearer " + token)
request.add_header("User-Agent", "")

74701


In [311]:
# This cell does internet stuff
# Send the request to Genius, and parse the response
response = urllib.request.urlopen(request, timeout=3)
string = response.read().decode('utf-8')
json_obj = json.loads(string)

In [312]:
# The data we want can be found here
# For this song there are 9 lyric/annotation pairs
referents = json_obj['response']['referents']
len(referents)

5

In [313]:
# Print each lyric along with its annotation
for ref in referents:
    print(ref['fragment'])
    print('************')
    print(recurse(ref['annotations'][0]['body']['dom']['children']))
    print()

Without past I can't disappoint
 My ancestry
************
She cannot disappoint anyone who she does not let know her, if you have no past then you are pure, if you have a past you definantly have something to feel ashamed in one form or another. 

[Verse]
************
The sample on this song seems to be the same as the one on  Baptism , but slightly altered. 

Catch a moth, hold it in my hand
 Crush it casually
************
The thoughts of someone so dissociated and desensitized to death that it doesn’t phase them 

We drown in pneumonia not
 Rivers and streams
************
Pneumonia is a human affliction. The illness is a symbol for any human suffering, forcing them from their true potential; eventually, we drown in that “illness”.   When people drown from pneumonia, their lungs fill with fluid and they cannot breathe, just like they would if they were drowning in water. 

Forever can I sleep inside
 Your menagerie
************
A menagerie being a place where a variety of animals/peop

In [243]:
# Function to recursively search the annotation dict
# and get the text, skipping blockquotes, formatting, images, etc.

def recurse(children):
    st = ''
    for child in children:
        #print(child)
        if type(child) == dict and 'tag' in child:
            if child['tag'] == 'p' or child['tag'] == 'a':
                st += recurse(child['children'])
            elif child['tag'] == 'blockquote':
                st += '[BLOCKQUOTE]'
        elif type(child) == list:
            st += recurse(child)
        elif type(child) == str:
            st += child + ' '
    return st

In [4]:
!grep -n '30800' artists.txt

128:30800,breakbot
